In [1]:
import requests
import pandas as pd
import utils.misc as misc
from config import *
import yahoo_fin.stock_info as si
from yahooquery import Ticker
import quandl
import os
from datetime import datetime, timedelta
from finvizfinance.screener.custom import Custom
import time
from glob import glob

In [2]:
BASE_URL = r"https://www.alphavantage.co/query?"
# av_api = 'CQLP7J9VZG14Y7O4'
av_api2 = 'IHLNUQ0G66C0MJID' #basil

In [3]:
request_times = [0] * 180

In [4]:
def av_api(func, api_key, symbol):
    assert symbol is not None
    symbol = symbol.strip().upper()

    url = f"{BASE_URL}function={func}&symbol={symbol}&apikey={api_key}"
    while True:
        if time.time() - request_times[-180] <= 60:
            time.sleep(1/3)
        else:
            response = requests.get(url)
            request_times.append(time.time())
            break
    try:
        if func == 'EARNINGS':
            return pd.DataFrame(response.json()['quarterlyEarnings'])
            # except:
            #     print(response.json())
            #     output = None
        elif func == 'BALANCE_SHEET':
            # print(response)
            # print(response.json())
            return pd.DataFrame(response.json()['quarterlyReports'])
        elif func == 'INCOME_STATEMENT':
            print(response.json())
            return pd.DataFrame(response.json()['quarterlyReports'])
    except:
        print(response.json())
        return 

In [5]:
def get_av_data(symbol, api_key):
    #For reported Earnings & Analyst Estimated EPS, totalRevenue, commonStockSharesOutstanding
    df1 = av_api('EARNINGS',api_key, symbol)
    # print(df1)
    df2 = av_api('INCOME_STATEMENT',api_key, symbol)[['fiscalDateEnding','totalRevenue']]
    print(df2)
    df1 = df1.merge(df2, on='fiscalDateEnding', how='left')
    df3 = av_api('BALANCE_SHEET',api_key, symbol)[['fiscalDateEnding','commonStockSharesOutstanding']]
    df1 = df1.merge(df3, on='fiscalDateEnding', how='left')

    return df1

In [ ]:
stocks = pd.read_csv('../data/remaining.csv', header=None)

In [ ]:
us_stocks = pd.read_csv('../data/us_stocks.csv')

In [ ]:
os.chdir(ep)

In [ ]:
os.chdir('data')

In [ ]:
os.getcwd()

In [ ]:
done_files = glob('*')

In [ ]:
for s in stocks[1]:
    outfile = fr'{s}_DATA.csv'
    industry = us_stocks[us_stocks['Ticker'] == s].iloc[0].Industry
    # print(str(industry))
    if (industry == 'Asset Management'):
        # print('Skipping asst mgmt', s)
        continue
    if outfile in done_files:
        # print('Already present', s)
        continue
    tries = 0
    # while True and tries < 5:
    #     tries += 1
    try:
        # print(s)
        df = get_av_data(s, av_api2)
        df.to_csv(outfile, mode='w', sep=',', header=df.columns)
        break
    except Exception as e:
        print("An exception occured: {}".format(e), s, industry)
        time.sleep(1/3)
        continue

In [ ]:
get_av_data('GRRR', av_api2)

In [ ]:
def get_ex_dates(symbol):
    symbol = 'META'
    tkr = Ticker(symbol)
    df = tkr.sec_filings
    df = df[(df['type'] == '10-Q') | ((df['type'] == '10-K'))]
    df.reset_index(inplace=True)
    df = df[['date','title']]
    df.rename(columns={'date': 'Ex-Date'}, inplace=True)
    return df

In [7]:
av_api('BALANCE_SHEET', av_api, 'META')

,fiscalDateEnding,reportedCurrency,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,...,currentLongTermDebt,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding
0,2022-12-31,USD,185727000000,59549000000,14681000000,40763000000,None,13466000000,40476000000,79518000000,...,None,9923000000,19923000000,18330000000,7764000000,125713000000,None,64799000000,0,2639000000
1,2022-09-30,USD,178894000000,58315000000,14308000000,69244000000,None,11227000000,40346000000,73738000000,...,None,9922000000,19922000000,17011000000,7504000000,124094000000,None,67056000000,0,2682000000
2,2022-06-30,USD,169779000000,55987000000,12681000000,68297000000,None,11525000000,39702000000,67588000000,...,None,None,16067000000,16402000000,7003000000,125767000000,None,69249000000,0,2704000000
3,2022-03-31,USD,164218000000,59265000000,14886000000,43890000000,None,11390000000,36630000000,61582000000,...,None,None,14053000000,16161000000,7010000000,123228000000,None,67712000000,0,2725000000
4,2021-12-31,USD,165987000000,66666000000,16601000000,47998000000,None,14039000000,34771000000,57809000000,...,None,0,13873000000,14108000000,7227000000,124879000000,None,69761000000,0,2764000000
5,2021-09-30,USD,169585000000,75421000000,14496000000,58075000000,None,12088000000,33680000000,53726000000,...,None,None,12640000000,14067000000,6859000000,133360000000,None,79233000000,0,2814000000
6,2021-06-30,USD,170609000000,80697000000,16186000000,64080000000,None,11698000000,32610000000,50909000000,...,None,None,12007000000,12459000000,6552000000,138227000000,None,85097000000,0,2834000000
7,2021-03-31,USD,163523000000,77322000000,19513000000,64219000000,None,10276000000,32139000000,47720000000,...,None,None,11614000000,10417000000,6575000000,133657000000,None,82343000000,0,2847000000
8,2020-12-31,USD,159316000000,75670000000,17576000000,61954000000,None,11335000000,31779000000,45633000000,...,None,None,10654000000,10207000000,6414000000,128290000000,None,77345000000,0,2854000000
9,2020-09-30,USD,146437000000,65799000000,11617000000,55620000000,None,8024000000,74474000000,42291000000,...,None,None,10616000000,9484000000,7121000000,117731000000,None,68513000000,0,2850000000


In [ ]:
import yfinance as yf
from yahoofinancials import YahooFinancials

In [ ]:
yahoo_financials = YahooFinancials('AAPL')
data = yahoo_financials.get_historical_price_data(start_date='2019-01-01', 
                                                  end_date='2019-12-31', 
                                                  time_interval='weekly')
aapl_df = pd.DataFrame(data['AAPL']['prices'])
aapl_df = aapl_df.drop('date', axis=1).set_index('formatted_date')
aapl_df.head()

In [ ]:
from __future__ import print_function
import time
import intrinio_sdk as intrinio
from intrinio_sdk.rest import ApiException

intrinio.ApiClient().set_api_key('YOUR_API_KEY')
intrinio.ApiClient().allow_retries(True)

identifier = 'AAPL'
page_size = 100
next_page = ''

response = intrinio.ZacksApi().get_zacks_analyst_ratings(identifier=identifier, page_size=page_size, next_page=next_page)
print(response)
    

##IGNORE FOR NOW

    NASDAQ DATALINK - FOR SHORT INTEREST

In [ ]:
ndl_api = 'dys7Lqv_HziWD1ZFk4ry'

In [ ]:
quandl.ApiConfig.api_key = ndl_api

In [ ]:
os.chdir('/Users/yash/Desktop/Trading/intern/Basil/finra_short_interest')

In [ ]:
df2 = pd.read_csv('equityshortinterest_2023_02_26_05_44_02.csv')

Q4 Earnings 2022

In [ ]:
df1.to_csv('us_stocks.csv')

In [ ]:
earnings_df = pd.DataFrame(columns=['Symbol', 'Ex-Date', 'title'])

In [ ]:
tickers = df1['Ticker']
for t in tickers:
    dft = get_ex_dates(t)
    dft["Date"] = dft["Ex-Date"].apply(
        lambda x: datetime.strptime(x, "%Y-%m-%d")
    )
    dft = dft[dft['Date'] >= datetime(2023,1,1)]
    if dft.shape[0] > 0:
        earnings_df.loc[len(earnings_df.index)] = [t]+list(dft.iloc[0,1:]) 

In [ ]:
import csv
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
CSV_URL = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=IBM&interval=1min&slice=year1month1&apikey={av_api}'

with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    for row in my_list:
        print(row)

FINNHUB

In [8]:
misc.install_package({'finnhub-python'})


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
